In [1]:
import pandas as pd
import numpy as np

In [11]:
train_df = pd.read_csv('std_data/MCTest/mc160/mc160.train.csv',index_col=0)
train_df.head()

,id,author,work_times,article,question_type,question,A,B,C,D,answer
0,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,multiple,What was the hardest thing for Tom and his fri...,Door,House,Window,Toilet,C
1,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,one,What did Tom need to fix first?,Door,House,Window,Toilet,D
2,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,multiple,Jim didn't bring which person with him?,Jim,Dolly,Molly,Holly,B
3,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,one,Who was Tom's best friend?,Molly,Holly,Jim,Dolly,C
4,mc160.train.1,88979267,12668,Lisa has a pet cat named Whiskers. Whiskers is...,multiple,What day is Whisker's Birthday?,Today,Last year,Friday,Saturday,D


In [8]:
import torch
from transformers import BertTokenizer, MobileBertForMultipleChoice
from torch.nn.modules import Softmax

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = MobileBertForMultipleChoice.from_pretrained('google/mobilebert-uncased')



Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MobileBertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MobileBertForMultipleChoice were not initiali

In [12]:
correct = 0

for i in range(1):
    fact_sent = train_df['article'][i]
    answers = [train_df[opt] for opt in ['A','B','C','D']]
    labels = torch.tensor(0).unsqueeze(0)

    fact_sent = [fact_sent for _ in range(len(answers)) ]

    encoding = tokenizer(list(zip(fact_sent, answers)), return_tensors='pt', padding=True)

    outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)

    loss, logits = outputs[:2]


    softmax = Softmax( dim=1)
    probabilities = softmax(logits)
    pred_asnwer = answers[np.argmax(probabilities.tolist()[0])]

    if pred_asnwer==ques_ans[i][0]['answer']:
        correct += 1
print(correct)

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.